# Tying it all Together

<img src="https://images-mm.s3.amazonaws.com/Big_Lebowski_Rug_Blue_Shirt_POP.jpg">

## PyMC3 is:

- a bunch of samplers
- a DSL to define models
- tools to assess sampling convergence
- tools to apply (posterior) samples

and has [great documentation](https://docs.pymc.io/)!

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls
import pymc3 as pm

from theano import shared

## A Toy Example: Fitting a Quadratic Function

Assume the function $f(x) = 2x^2 + 2x - 10$:

In [ ]:
x = np.linspace(-4, 4, 100)

In [ ]:
def f(x):
    return 2*x**2 + 2*x - 10

In [ ]:
fig = go.FigureWidget(
    data=[
        go.Scatter(x=x, y=f(x), opacity=0.5, showlegend=False)
    ],
    layout={
        'width': 800,
        'height': 800
    }
)

fig

Pretending not to know the real function, let's generate 2 clusters of "observed" data:

In [ ]:
x_observed = np.concatenate((
    np.random.normal(loc=-2, scale=0.1, size=100),
    np.random.normal(loc=1, scale=0.1, size=100)
))
x_observed_shared = shared(x_observed)  # a Theano shared variable
y_observed = np.random.normal(loc=f(x_observed), scale=1)

fig.add_scatter(x=x_observed, y=y_observed, mode='markers', opacity=0.8, showlegend=False);

In [ ]:
with pm.Model() as non_linear_model:
    # priors
    b0 = pm.Normal('b0', mu=0, sd=100)
    b1 = pm.Normal('b1', mu=1, sd=100)
    b2 = pm.Normal('b2', mu=1, sd=100)
    sigma = pm.HalfNormal('sigma', sd=1)
    
    mu = b0 + b1*x_observed_shared + b2*x_observed_shared**2
    
    # likelihood
    y_likelihood = pm.Normal('y_likelihood', mu=mu, sd=sigma, observed=y_observed)

In [ ]:
with non_linear_model:
    trace = pm.sample(2000, tune=1000)

What's that trace object?

In [ ]:
pm.summary(trace)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.autocorrplot(trace);

Not sure why this convenient util is hidden so far away...

In [ ]:
trace_df = pm.backends.tracetab.trace_to_dataframe(trace).drop('sigma', axis=1)

## Testing the Fit: Making Some Predictions

In [ ]:
x_test = np.linspace(-4, 4, 10)

In [ ]:
xs = np.array([-2, 0, 2, 4])
models = np.array([[1, 2, 1], [0.5, 2.5, 1], [1.5, 1.5, 1], [0.8, 2.2, 1]])

In [ ]:
def yhat(m, X=None):
    return m[0] + m[1]*X + m[2]*X**2

In [ ]:
def apply_model(x, model):
    return model.iloc[0].b0 + model.iloc[0].b1*x + model.iloc[0].b2*x**2

In [ ]:
for _ in range(100):
    random_model = trace_df.sample(n=1)
    fig.add_scatter(x=x_test, y=[apply_model(x, random_model) for x in x_test],
                    mode='lines', opacity=0.1, line={'width': 0.5, 'color': 'grey', 'shape': 'spline'}, showlegend=False);

In [ ]:
fig

In [ ]:
x_observed_shared.set_value(x_test)

In [ ]:
ppc = pm.sample_ppc(trace=trace, model=non_linear_model, size=100)

In [ ]:
hpd_min4 = np.apply_along_axis(pm.hpd, arr=ppc['y_likelihood'][:,:,0], axis=1)
hpd_0 = np.apply_along_axis(pm.hpd, arr=ppc['y_likelihood'][:,:,1], axis=1)
hpd_2 = np.apply_along_axis(pm.hpd, arr=ppc['y_likelihood'][:,:,2], axis=1)

In [ ]:
def x_hpd(X):
    return pm.hpd(X.flatten())

In [ ]:
y_test = ppc['y_likelihood'].reshape(-1, len(x_test))

In [ ]:
y_test.shape

In [ ]:
hpd_all = np.apply_along_axis(pm.hpd, arr=y_test, axis=0)

In [ ]:
hpd_all.shape

In [ ]:
pm.hpd(y_test).shape

In [ ]:
hpd_all

In [ ]:
fig.add_scatter(x=x_test, y=hpd_all[1], name='high', mode='lines', line={'shape': 'spline'})

In [ ]:
fig.add_scatter(x=x_test, y=hpd_all[0], name='low', mode='lines', line={'shape': 'spline'})